<div style="line-height:0.4">
<h1 style="color:#0FCBC6"> PySpark 1: Basic notions </h1>
<span style="display: inline-block;">
    <h3 style="color: lightblue; display: inline;">Keywords:</h3> Dataframe operations + Files + RDD
</span>
</div>

In [56]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import isnan, col, when, rand
from pyspark.ml.feature import Imputer
from pyspark import SparkContext

Create a `SparkSession` with the `SparkSession.builder`
=> Set various configuration options and attributes to customize its behavior.

1. **`appName`:** Specifies the name of your Spark application. This name will appear in the Spark UI and logs.

2. **`master`:** Specifies the cluster manager or cluster URL to connect to. You can set this to "local" for local testing, "yarn" for YARN cluster, or a specific URL for other cluster managers.

3. **`config`:** Sets additional configuration options using a dictionary.

4. **`config("spark.some.option")` :** Sets specific Spark configuration options using the `config` method.      
&emsp;&emsp;=> `config("spark.executor.memory", "2g")` Sets the amount of memory per executor.

5. **`enableHiveSupport`:** Enables access to "Hive features".

6. **`getOrCreate()`:** Retrieves an existing `SparkSession` or creates a new one if it doesn't exist.

7. **Others:** 
   - `spark.executor.cores`: The number of cores to use on each executor.
   - `spark.driver.memory`: The amount of memory to allocate to the driver.
   - `spark.sql.shuffle.partitions`: The number of partitions to use when shuffling data in Spark SQL operations.
   - `spark.default.parallelism`: The default level of parallelism to use for RDDs and DataFrames.
   - `spark.serializer`: The serializer used for serializing data. (Default: org.apache.spark.serializer.JavaSerializer)
   - `spark.logConf`: Controls whether Spark configuration is logged when the `SparkSession` is created.

<h2 style="color:#0FCBC6"><u>Example 1</u></h2>

In [3]:
# Create a Spark session with custom configurations
spark = SparkSession.builder \
    .appName("CustomSparkSession") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "4") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()


23/08/15 18:02:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
""" N.B. Stop the Spark session when you're done! 
The spark.stop() method gracefully shuts down the Spark session, releases resources, and terminates any running Spark jobs. 
To prevent resource leakage especially when working in an interactive environment like Jupyter Notebook.
"""
#spark.stop()

In [2]:
spark = SparkSession.builder \
    .appName("PySparkTutorial") \
    .getOrCreate()

23/08/15 17:46:03 WARN Utils: Your hostname, hpmint resolves to a loopback address: 127.0.1.1; using 192.168.1.81 instead (on interface eno1)
23/08/15 17:46:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 17:46:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
# Load data from a CSV file
data_path = "./datasets_for_pyspark/naggfs.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [11]:
""" Load data from JSON """
# Read 
json_path = "./datasets_for_pyspark/for_tutorial_1/train_small.json"
"""
# To create the Dataframe
# => using df = spark.read.json(json_path) is not enough 
# Add 'multiline' option avoid the error:  Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the referenced columns only include the internal corrupt record column
"""
# Dataframe
df_j = spark.read.option("multiline", "true").option("mode", "PERMISSIVE").json(json_path)
df_j.show()

+--------------------+--------------------+------+-----+--------------------+
|                 du1|                 du2|    id|label|             message|
+--------------------+--------------------+------+-----+--------------------+
| i honeslty never...| i would make it ...|2_2374|    N| < i honeslty nev...|
| i honeslty never...|                 18 |2_2375|    N| < i honeslty nev...|
| i honeslty never...|    but here i am , |2_2376|    D| < i honeslty nev...|
| i would make it ...|                 18 |2_2377|    N|i honeslty never ...|
| i would make it ...|    but here i am , |2_2378|    D|i honeslty never ...|
| i would make it ...| living it up on ...|2_2379|    C|i honeslty never ...|
|                 18 |    but here i am , |2_2380|    N|i honeslty never ...|
|                 18 | living it up on ...|2_2381|    N|i honeslty never ...|
|                 18 |           2020👌🏼 |2_2382|    N|i honeslty never ...|
|    but here i am , | living it up on ...|2_2383|    C|i honeslty

In [ ]:
# Writing Data
output_path = "./datasets_for_pyspark/output_created_can_be_cancelled.csv"
df.write.csv(output_path, header=True)

In [12]:
df_j

DataFrame[du1: string, du2: string, id: string, label: string, message: string]

<h3 style="color:#0FCBC6"> => Display DataFrame and Schema </h3>

In [15]:
df.show()

+----------------+-------+----------+-----------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|Series_reference| Period|Data_value|     STATUS|  UNITS|MAGNTUDE|             Subject|               Group|      Series_title_1|Series_title_2|Series_title_3|Series_title_4|Series_title_5|
+----------------+-------+----------+-----------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|GFSA.SGS01G01Z90|2009.06|      3881|    REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null|          null|          null|
|GFSA.SGS01G01Z90|2010.06|     -3356|    REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null|          null|          null|
|GFSA.SGS01G01Z90|2011.06|    -13181|    REVISED|D

In [16]:
df.printSchema()

root
 |-- Series_reference: string (nullable = true)
 |-- Period: double (nullable = true)
 |-- Data_value: integer (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- UNITS: string (nullable = true)
 |-- MAGNTUDE: integer (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Series_title_1: string (nullable = true)
 |-- Series_title_2: string (nullable = true)
 |-- Series_title_3: string (nullable = true)
 |-- Series_title_4: string (nullable = true)
 |-- Series_title_5: string (nullable = true)



<h3 style="color:#0FCBC6"> => Columns and Rows </h3>

In [18]:
df.select("STATUS")

DataFrame[STATUS: string]

<h3 style="color:#0FCBC6"> => Grouping and Aggregation </h3>

In [19]:
df.groupBy("MAGNTUDE").agg({"MAGNTUDE": "avg"})

DataFrame[MAGNTUDE: int, avg(MAGNTUDE): double]

<h3 style="color:#0FCBC6"> => Transformations </h3>

In [20]:
"""" Add Columns """
df.withColumn("new_column", df["MAGNTUDE"] + 5)


DataFrame[Series_reference: string, Period: double, Data_value: int, STATUS: string, UNITS: string, MAGNTUDE: int, Subject: string, Group: string, Series_title_1: string, Series_title_2: string, Series_title_3: string, Series_title_4: string, Series_title_5: string, new_column: int]

In [21]:
""" Rename Columns """
df.withColumnRenamed("new_column", "MAGNITUDE_ADDED_LATER")

DataFrame[Series_reference: string, Period: double, Data_value: int, STATUS: string, UNITS: string, MAGNTUDE: int, Subject: string, Group: string, Series_title_1: string, Series_title_2: string, Series_title_3: string, Series_title_4: string, Series_title_5: string]

In [22]:
""" Drop Columns """
df.drop("MAGNITUDE_ADDED_LATER")

DataFrame[Series_reference: string, Period: double, Data_value: int, STATUS: string, UNITS: string, MAGNTUDE: int, Subject: string, Group: string, Series_title_1: string, Series_title_2: string, Series_title_3: string, Series_title_4: string, Series_title_5: string]

<h3 style="color:#0FCBC6"> => SQL Queries</h3>

In [26]:
df.createOrReplaceTempView("table_name")

In [31]:
result_df = spark.sql("SELECT * FROM table_name WHERE Data_value > 2000")
print(result_df)
print()
result_df.show(n=10)

DataFrame[Series_reference: string, Period: double, Data_value: int, STATUS: string, UNITS: string, MAGNTUDE: int, Subject: string, Group: string, Series_title_1: string, Series_title_2: string, Series_title_3: string, Series_title_4: string, Series_title_5: string]

+----------------+-------+----------+-------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|Series_reference| Period|Data_value| STATUS|  UNITS|MAGNTUDE|             Subject|               Group|      Series_title_1|Series_title_2|Series_title_3|Series_title_4|Series_title_5|
+----------------+-------+----------+-------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|GFSA.SGS01G01Z90|2009.06|      3881|REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null|          null|    

<h3 style="color:#0FCBC6"> => Handling Missing Data </h3>

In [42]:
""" Insert NaN values randomly """

# Specify the probability of NaN insertion
nan_probability = 0.2
df_with_nan = df.withColumn("UNITS", when(rand() < nan_probability, None).otherwise(col("UNITS")))
df_with_nan.show()

+----------------+-------+----------+-----------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|Series_reference| Period|Data_value|     STATUS|  UNITS|MAGNTUDE|             Subject|               Group|      Series_title_1|Series_title_2|Series_title_3|Series_title_4|Series_title_5|
+----------------+-------+----------+-----------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|GFSA.SGS01G01Z90|2009.06|      3881|    REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null|          null|          null|
|GFSA.SGS01G01Z90|2010.06|     -3356|    REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null|          null|          null|
|GFSA.SGS01G01Z90|2011.06|    -13181|    REVISED|D

In [44]:
# Search for NaN values
nan_columns = [col_name for col_name in df_with_nan.columns if df.select(isnan(col(col_name))).collect()[0][0]]
print("Columns with NaN values:", nan_columns)

Columns with NaN values: []


In [47]:
# Identify columns with Null values
nan_columns = [col_name for col_name in df_with_nan.columns if df.filter(col(col_name).isNull()).count() > 0]
print("Columns with NaN values:", nan_columns)

Columns with NaN values: ['Series_title_2', 'Series_title_3', 'Series_title_4', 'Series_title_5']


In [55]:
""" Options: 
df.dropna()
df.fillna(0)
"""
#df.dropna()
#df.fillna(0)

' Options: \ndf.dropna()\ndf.fillna(0)\n'

In [ ]:
""" Imputer for numbers """
# N.B. => IllegalArgumentException: requirement failed: Column Series_title_2 must be of type numeric but was actually of type string.
# imputer = Imputer(inputCols=["Series_title_2"], outputCols=["Series_title_4"])
# df_imputed = imputer.fit(df_with_nan).transform(df_with_nan)

In [51]:
""" Imputer for strings """
column_to_impute = "Series_title_4"
default_value = "default_value"
imputed_df = df.withColumn(column_to_impute, when(col(column_to_impute).isNull(), default_value).otherwise(col(column_to_impute)))
imputed_df.show()

+----------------+-------+----------+-----------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|Series_reference| Period|Data_value|     STATUS|  UNITS|MAGNTUDE|             Subject|               Group|      Series_title_1|Series_title_2|Series_title_3|Series_title_4|Series_title_5|
+----------------+-------+----------+-----------+-------+--------+--------------------+--------------------+--------------------+--------------+--------------+--------------+--------------+
|GFSA.SGS01G01Z90|2009.06|      3881|    REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null| default_value|          null|
|GFSA.SGS01G01Z90|2010.06|     -3356|    REVISED|Dollars|       6|Government Financ...|General Governmen...|Net operating bal...|          null|          null| default_value|          null|
|GFSA.SGS01G01Z90|2011.06|    -13181|    REVISED|D

<h3 style="color:#0FCBC6"> => Create dataframes </h3>

In [52]:
# Create a list of raw objects
data = [
    Row(col1=1, col2="A", col3=10.5),
    Row(col1=2, col2="B", col3=20.0),
    Row(col1=3, col2="C", col3=15.3),
    Row(col1=4, col2="D", col3=8.7),
    Row(col1=5, col2="E", col3=12.2),
    Row(col1=6, col2="F", col3=18.9),
    Row(col1=7, col2="G", col3=25.1),
    Row(col1=8, col2="H", col3=9.4),
    Row(col1=9, col2="I", col3=14.7),
    Row(col1=10, col2="J", col3=22.3)
]

# DataFrame from list 
df = spark.createDataFrame(data)

df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   A|10.5|
|   2|   B|20.0|
|   3|   C|15.3|
|   4|   D| 8.7|
|   5|   E|12.2|
|   6|   F|18.9|
|   7|   G|25.1|
|   8|   H| 9.4|
|   9|   I|14.7|
|  10|   J|22.3|
+----+----+----+



<h2 style="color:#0FCBC6"><u>Example 2</u></h2>

<h3 style="color:#0FCBC6"> => Files </h3>

In [69]:
# Stop the SparkSession before creating the SparkContext 
spark.stop()
# Stop the current session, to run again this cell
sc.stop()

# Create SparkContext object 
sc = SparkContext(appName="FileHandling")

# Read a text file
pride_rdd = sc.textFile("./datasets_for_pyspark/text_files/pride_and_prejudice.txt")
moby_dick_rdd = sc.textFile("./datasets_for_pyspark/text_files/moby_dick.txt")
alice_rdd = sc.textFile("./datasets_for_pyspark/text_files/alice.txt")


# Show the content of RDDs
print("Pride and Prejudice:")
print(pride_rdd.collect())
print()
print("\nMoby-Dick:")
print(moby_dick_rdd.collect())
print()
print("\nAlice's Adventures in Wonderland:")
print(alice_rdd.collect())

Pride and Prejudice:
['It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.']


Moby-Dick:
['Call me Ishmael. Some years ago - never mind how long precisely - having little or no money in my purse, and nothing particular to interest me on shore...', '']


Alice's Adventures in Wonderland:
['Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do. Once or twice she had peeped into the book her sister was reading...', '']


In [70]:
# Read multiple text files as key-value pairs
kv_rdd = sc.wholeTextFiles("./datasets_for_pyspark/text_files")
kv_rdd.collect()

[('file:/home/notto4/Desktop/coding_trials_drafts/AI_ML_DL/PySpark_tutorials/datasets_for_pyspark/text_files/pride_and_prejudice.txt',
  'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\n'),
 ('file:/home/notto4/Desktop/coding_trials_drafts/AI_ML_DL/PySpark_tutorials/datasets_for_pyspark/text_files/moby_dick.txt',
  'Call me Ishmael. Some years ago - never mind how long precisely - having little or no money in my purse, and nothing particular to interest me on shore...\n\n'),
 ('file:/home/notto4/Desktop/coding_trials_drafts/AI_ML_DL/PySpark_tutorials/datasets_for_pyspark/text_files/alice.txt',
  'Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do. Once or twice she had peeped into the book her sister was reading...\n\n')]

In [71]:
# Count the number of lines in each RDD
pride_lines = pride_rdd.count()
moby_dick_lines = moby_dick_rdd.count()
alice_lines = alice_rdd.count()

print("\nNumber of lines:")
print("Pride and Prejudice:", pride_lines)
print("Moby-Dick:", moby_dick_lines)
print("Alice's Adventures in Wonderland:", alice_lines)


Number of lines:
Pride and Prejudice: 1
Moby-Dick: 2
Alice's Adventures in Wonderland: 2


In [72]:
# Actions: Find the longest line in each RDD
longest_line_pride = pride_rdd.max(key=lambda line: len(line))
longest_line_moby_dick = moby_dick_rdd.max(key=lambda line: len(line))
longest_line_alice = alice_rdd.max(key=lambda line: len(line))

print("\nLongest Lines:")
print("Pride and Prejudice:", longest_line_pride)
print("Moby-Dick:", longest_line_moby_dick)
print("Alice's Adventures in Wonderland:", longest_line_alice)


Longest Lines:
Pride and Prejudice: It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.
Moby-Dick: Call me Ishmael. Some years ago - never mind how long precisely - having little or no money in my purse, and nothing particular to interest me on shore...
Alice's Adventures in Wonderland: Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do. Once or twice she had peeped into the book her sister was reading...


<h3 style="color:#0FCBC6"> => Recap on RDD </h3>    
<div style="margin-top: -15px;">

A Resilient Distributed Dataset (RDD), is the fundamental data structure in Spark (the basic abstraction).      
RDDs represent an immutable, partitioned collection of elements (data) that can be processed in parallel across a cluster of computers.    
RDDs provide an abstraction layer that allows you to perform transformations and actions on distributed data without worrying <br> about the low-level details of parallel processing and fault recovery.

2 types of operations:
<div style="margin-top: -15px;">

- Transformations: These are operations that create a new RDD from an existing one. <br> 
Transformations are performed lazily and build up a sequence of transformations in the lineage.
- Actions: These are operations that return a value to the driver program or write data to an external storage system.          
Actions trigger the actual execution of the transformations and bring data from distributed nodes back to the driver program.           
</div>

In [67]:
def process_number(number):
    """ Perform an action using the RDD and accumulator. """
    global accumulator
    accumulator += number
    return number * 2

try:
    # Create an RDD from a list of numbers
    numbers_rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    print("Number of partitions:", numbers_rdd.getNumPartitions())

    # Create a broadcast variable
    broadcast_var = sc.broadcast({"key": "value"})
    # Create an accumulator
    accumulator = sc.accumulator(0)

    processed_numbers_rdd = numbers_rdd.map(process_number)
    total = processed_numbers_rdd.sum()

    print("Processed Numbers:", processed_numbers_rdd.collect())
    print("Total:", total)
    print("Accumulator Value:", accumulator.value)

finally:
    # Stop the Spark context
    sc.stop()

Number of partitions: 4


Processed Numbers: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
Total: 110
Accumulator Value: 110
